# 에이전트 메모리에 의미론적 검색을 추가하는 방법

이 가이드는 에이전트의 메모리 저장소에서 의미론적 검색을 활성화하는 방법을 보여줍니다. 이를 통해 의미론적 유사성으로 저장소의 항목을 검색할 수 있습니다.

!!! tip 필수 조건
    이 가이드는 [LangGraph의 메모리](https://langchain-ai.github.io/langgraph/concepts/memory/)에 대한 익숙함을 가정합니다.

먼저 이 가이드의 필수 조건을 설치합니다.

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai langchain

In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

다음으로 [인덱스 구성](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.IndexConfig)으로 저장소를 만듭니다. 기본적으로 저장소는 의미론적/벡터 검색 없이 구성됩니다. 저장소를 생성할 때 [IndexConfig](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.IndexConfig)를 저장소의 생성자에 제공하여 항목 인덱싱을 선택할 수 있습니다. 저장소 클래스가 이 인터페이스를 구현하지 않거나 인덱스 구성을 전달하지 않으면 의미론적 검색이 비활성화되고 `put` 또는 `aput`에 전달된 모든 `index` 인수는 효과가 없습니다. 아래는 예제입니다.

In [ ]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

# 의미론적 검색이 활성화된 저장소 생성
embeddings = init_embeddings("openai:text-embedding-3-small")
store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
    }
)

이제 몇 가지 메모리를 저장해 봅시다:

In [ ]:
# 일부 메모리 저장
store.put(("user_123", "memories"), "1", {"text": "I love pizza"})
store.put(("user_123", "memories"), "2", {"text": "I prefer Italian food"})
store.put(("user_123", "memories"), "3", {"text": "I don't like spicy food"})
store.put(("user_123", "memories"), "3", {"text": "I am studying econometrics"})
store.put(("user_123", "memories"), "3", {"text": "I am a plumber"})

자연어를 사용하여 메모리를 검색합니다:

In [ ]:
# 음식 선호도에 대한 메모리 찾기
memories = store.search(("user_123", "memories"), query="I like food?", limit=5)

for memory in memories:
    print(f"Memory: {memory.value['text']} (similarity: {memory.score})")

## 에이전트에서 사용하기

저장소를 주입하여 모든 노드에 의미론적 검색을 추가합니다.

In [ ]:
from typing import Optional

from langchain.chat_models import init_chat_model
from langgraph.store.base import BaseStore

from langgraph.graph import START, MessagesState, StateGraph

llm = init_chat_model("openai:gpt-4o-mini")


def chat(state, *, store: BaseStore):
    # 사용자의 마지막 메시지를 기반으로 검색
    items = store.search(
        ("user_123", "memories"), query=state["messages"][-1].content, limit=2
    )
    memories = "\n".join(item.value["text"] for item in items)
    memories = f"## Memories of user\n{memories}" if memories else ""
    response = llm.invoke(
        [
            {"role": "system", "content": f"You are a helpful assistant.\n{memories}"},
            *state["messages"],
        ]
    )
    return {"messages": [response]}


builder = StateGraph(MessagesState)
builder.add_node(chat)
builder.add_edge(START, "chat")
graph = builder.compile(store=store)

for message, metadata in graph.stream(
    input={"messages": [{"role": "user", "content": "I'm hungry"}]},
    stream_mode="messages",
):
    print(message.content, end="")

## `create_react_agent`에서 사용하기 {#using-in-create-react-agent}

`prompt` 함수에 저장소를 주입하여 도구 호출 에이전트에 의미론적 검색을 추가합니다. 에이전트가 수동으로 메모리를 저장하거나 검색할 수 있도록 도구에서 저장소를 사용할 수도 있습니다.

In [ ]:
import uuid
from typing import Optional

from langchain.chat_models import init_chat_model
from langgraph.prebuilt import InjectedStore
from langgraph.store.base import BaseStore
from typing_extensions import Annotated

from langgraph.prebuilt import create_react_agent


def prepare_messages(state, *, store: BaseStore):
    # 사용자의 마지막 메시지를 기반으로 검색
    items = store.search(
        ("user_123", "memories"), query=state["messages"][-1].content, limit=2
    )
    memories = "\n".join(item.value["text"] for item in items)
    memories = f"## Memories of user\n{memories}" if memories else ""
    return [
        {"role": "system", "content": f"You are a helpful assistant.\n{memories}"}
    ] + state["messages"]


# 도구 내에서 저장소를 직접 사용할 수도 있습니다!
def upsert_memory(
    content: str,
    *,
    memory_id: Optional[uuid.UUID] = None,
    store: Annotated[BaseStore, InjectedStore],
):
    """데이터베이스에 메모리를 업서트합니다."""
    # LLM은 이 도구를 사용하여 새 메모리를 저장할 수 있습니다
    mem_id = memory_id or uuid.uuid4()
    store.put(
        ("user_123", "memories"),
        key=str(mem_id),
        value={"text": content},
    )
    return f"Stored memory {mem_id}"


agent = create_react_agent(
    init_chat_model("openai:gpt-4o-mini"),
    tools=[upsert_memory],
    # 'prompt' 함수는 LLM용 메시지를 준비하기 위해 실행됩니다. 각 LLM 호출 직전에 호출됩니다
    # 각 LLM 호출 직전에 호출됩니다
    prompt=prepare_messages,
    store=store,
)

In [7]:
for message, metadata in agent.stream(
    input={"messages": [{"role": "user", "content": "I'm hungry"}]},
    stream_mode="messages",
):
    print(message.content, end="")

What are you in the mood for? Since you love Italian food and pizza, maybe something in that realm would be great! Would you like suggestions for a specific dish or restaurant?

## 고급 사용법

#### 다중 벡터 인덱싱

메모리의 다양한 측면을 개별적으로 저장하고 검색하여 리콜을 개선하거나 특정 필드를 인덱싱에서 제외합니다.

In [ ]:
# 메모리 내용과 감정적 맥락을 모두 임베딩하도록 저장소 구성
store = InMemoryStore(
    index={"embed": embeddings, "dims": 1536, "fields": ["memory", "emotional_context"]}
)
# 다양한 내용/감정 쌍으로 메모리 저장
store.put(
    ("user_123", "memories"),
    "mem1",
    {
        "memory": "Had pizza with friends at Mario's",
        "emotional_context": "felt happy and connected",
        "this_isnt_indexed": "I prefer ravioli though",
    },
)
store.put(
    ("user_123", "memories"),
    "mem2",
    {
        "memory": "Ate alone at home",
        "emotional_context": "felt a bit lonely",
        "this_isnt_indexed": "I like pie",
    },
)

# 감정 상태에 초점을 맞춘 검색 - mem2와 일치
results = store.search(
    ("user_123", "memories"), query="times they felt isolated", limit=1
)
print("Expect mem 2")
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Emotion: {r.value['emotional_context']}\n")

# 사교적 식사에 초점을 맞춘 검색 - mem1과 일치
print("Expect mem1")
results = store.search(("user_123", "memories"), query="fun pizza", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Emotion: {r.value['emotional_context']}\n")

print("Expect random lower score (ravioli not indexed)")
results = store.search(("user_123", "memories"), query="ravioli", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Emotion: {r.value['emotional_context']}\n")

#### 저장 시간에 필드 재정의
저장소의 기본 구성에 관계없이 `put(..., index=[...fields])`를 사용하여 특정 메모리를 저장할 때 임베드할 필드를 재정의할 수 있습니다.

In [ ]:
store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
        "fields": ["memory"],
    }  # 기본적으로 memory 필드를 임베드
)

# 기본 인덱싱으로 하나의 메모리 저장
store.put(
    ("user_123", "memories"),
    "mem1",
    {"memory": "I love spicy food", "context": "At a Thai restaurant"},
)

# 임베드할 필드를 재정의하여 다른 메모리 저장
store.put(
    ("user_123", "memories"),
    "mem2",
    {"memory": "The restaurant was too loud", "context": "Dinner at an Italian place"},
    index=["context"],  # 재정의: context만 임베드
)

# 음식에 대한 검색 - mem1과 일치 (기본 필드 사용)
print("Expect mem1")
results = store.search(
    ("user_123", "memories"), query="what food do they like", limit=1
)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Context: {r.value['context']}\n")

# 레스토랑 분위기에 대한 검색 - mem2와 일치 (재정의된 필드 사용)
print("Expect mem2")
results = store.search(
    ("user_123", "memories"), query="restaurant environment", limit=1
)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Context: {r.value['context']}\n")

#### 특정 메모리에 대한 인덱싱 비활성화

일부 메모리는 내용으로 검색할 수 없어야 합니다. `put(..., index=False)`를 사용하여 이러한 메모리를 저장하면서 인덱싱을 비활성화할 수 있습니다. 예제:

In [ ]:
store = InMemoryStore(index={"embed": embeddings, "dims": 1536, "fields": ["memory"]})

# 일반 인덱스된 메모리 저장
store.put(
    ("user_123", "memories"),
    "mem1",
    {"memory": "I love chocolate ice cream", "type": "preference"},
)

# 인덱싱 없이 시스템 메모리 저장
store.put(
    ("user_123", "memories"),
    "mem2",
    {"memory": "User completed onboarding", "type": "system"},
    index=False,  # 인덱싱 완전히 비활성화
)

# 음식 선호도에 대한 검색 - mem1 찾기
print("Expect mem1")
results = store.search(("user_123", "memories"), query="what food preferences", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Type: {r.value['type']}\n")

# 온보딩에 대한 검색 - mem2를 찾지 못함 (인덱스되지 않음)
print("Expect low score (mem2 not indexed)")
results = store.search(("user_123", "memories"), query="onboarding status", limit=1)
for r in results:
    print(f"Item: {r.key}; Score ({r.score})")
    print(f"Memory: {r.value['memory']}")
    print(f"Type: {r.value['type']}\n")